In [114]:
# %%capture
# !pip install boto3

In [115]:
import boto3
import random
import os
from dotenv import load_dotenv
import time

load_dotenv()

AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
REGION_NAME = os.getenv('REGION_NAME')
AGENT_ID = os.getenv('AGENT_ID')
AGENT_ALIAS_ID = os.getenv('AGENT_ALIAS_ID')

bedrock_agent_runtime = boto3.client(
                'bedrock-agent-runtime',
                aws_access_key_id = AWS_ACCESS_KEY_ID,
                aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
                region_name= REGION_NAME
            )
session = random.randint(0,9999) #setting a random sessionID

## Below shows how to call the agents that are created in AWS bedrock

In [116]:
## Function for Invoking the agent (lead_gen) that is created in AWS bedrock

def generate_response_query(input_text,agent_id,agent_alias_id,session_id):
  response = bedrock_agent_runtime.invoke_agent(inputText=input_text,
      agentId=agent_id,
      agentAliasId=agent_alias_id,
      sessionId=session_id,
  )

  print(response.get('completion'))

  # Response returned from the agent

  event_stream = response.get('completion')

  for event in event_stream:
    if 'chunk' in event:
      data = event['chunk']['bytes']
      print(f"Final answer ->\n{data.decode('utf8')}")

In [117]:
input_text:str = "Create a lead as Mahesh Yadav, California, 339111222333" # replace this with a prompt relevant to your agent
agent_id:str = AGENT_ID # note this from the agent console on Bedrock
agent_alias_id:str = AGENT_ALIAS_ID # fixed for draft version of the agent
session_id:str =  str(session) # random identifier

In [118]:
generate_response_query(input_text,agent_id,agent_alias_id,session_id)

Final answer ->
Lead created successfully for Mahesh Yadav in California with phone number 339111222333.


## Creating an Agent and Invoking the Lambda functions stored in AWS

In [119]:
# Note: Here bedrock-agent is used instead of bedrock-agent-runtime
# Bedrock-agent has create_agent() function, while bedrock-agent-runtime has invoke_agent()

bedrock_agent = boto3.client(
                'bedrock-agent',
                aws_access_key_id = AWS_ACCESS_KEY_ID,
                aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
                region_name= REGION_NAME
            )

In [120]:
aws_agent = bedrock_agent.create_agent(
    agentName='Lead_gen_notebook',
    description='Extract name, phone number and address for lead generation. Or extract address and monthly bill for calculating solar potential',
    foundationModel='anthropic.claude-v2:1',
    instruction="""You are a solar panel financial advisor, your job is to extract address as string and monthly bills as integer for calling functions to calculate solar potential. 
    You will return the detailed solar potential calculations to the user descriptive format along with a question in the end as "Please provide your name, phone number and address for contacting you".
    If the user provides name, phone number and address, you should call create lead function with these information for generating leads. 
    Strictly respond to the question for calling the function that is required to answer.""",
    agentResourceRoleArn='arn:aws:iam::674564987265:role/service-role/AmazonBedrockExecutionRoleForAgents_KT4F6CHPJV'
)

In [121]:
time.sleep(4)
#Creating the action group for the agent
response = bedrock_agent.create_agent_action_group(
    agentId=aws_agent['agent'].get('agentId'),
    agentVersion="DRAFT",
    actionGroupName='LeadGenAction',
    actionGroupExecutor={
        'lambda': 'arn:aws:lambda:us-east-1:674564987265:function:lead_gen_solar'
    },
    apiSchema={
        's3': {
            's3BucketName': 'legalgraphbucket',
            's3ObjectKey': 'lead_generation.json'
        }
    },
    actionGroupState='ENABLED'
)
print(response)

{'ResponseMetadata': {'RequestId': 'cfe5f716-2047-4712-be78-e98ed45b9ea1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 24 Jan 2024 10:22:03 GMT', 'content-type': 'application/json', 'content-length': '439', 'connection': 'keep-alive', 'x-amzn-requestid': 'cfe5f716-2047-4712-be78-e98ed45b9ea1', 'x-amz-apigw-id': 'SCivuED5IAMEsqQ=', 'x-amzn-trace-id': 'Root=1-65b0e4ca-61f576f604c5e23b607c71a5'}, 'RetryAttempts': 0}, 'agentActionGroup': {'agentId': 'PHEBBOQFZA', 'agentVersion': 'DRAFT', 'actionGroupId': '7RF7FA8L8R', 'actionGroupName': 'LeadGenAction', 'createdAt': datetime.datetime(2024, 1, 24, 10, 22, 3, 323339, tzinfo=tzutc()), 'updatedAt': datetime.datetime(2024, 1, 24, 10, 22, 3, 323339, tzinfo=tzutc()), 'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:674564987265:function:lead_gen_solar'}, 'apiSchema': {'s3': {'s3BucketName': 'legalgraphbucket', 's3ObjectKey': 'lead_generation.json'}}, 'actionGroupState': 'ENABLED'}}


In [122]:
# Check status of the agent

response = bedrock_agent.get_agent(
    agentId=aws_agent['agent'].get('agentId')
)
print(response['agent'].get('agentStatus'))

NOT_PREPARED


In [123]:
#Preparing the agent

response = bedrock_agent.prepare_agent(
    agentId=aws_agent['agent'].get('agentId')
)

In [124]:
time.sleep(4)
# Check status of the agent again

response = bedrock_agent.get_agent(
    agentId=aws_agent['agent'].get('agentId')
)
print(response['agent'].get('agentStatus'))

PREPARED


In [125]:
# Creating the alias
agent_alias = bedrock_agent.create_agent_alias(
    agentId=aws_agent['agent'].get('agentId'),
    agentAliasName='Dummy',
)
print(agent_alias)

{'ResponseMetadata': {'RequestId': 'fb652289-ff2d-4239-b0e7-38f147f49aec', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Wed, 24 Jan 2024 10:22:12 GMT', 'content-type': 'application/json', 'content-length': '329', 'connection': 'keep-alive', 'x-amzn-requestid': 'fb652289-ff2d-4239-b0e7-38f147f49aec', 'x-amz-apigw-id': 'SCixOGt3oAMEiqA=', 'x-amzn-trace-id': 'Root=1-65b0e4d4-34ce3e8d4458428f55bd2c96'}, 'RetryAttempts': 0}, 'agentAlias': {'agentId': 'PHEBBOQFZA', 'agentAliasId': 'UICETT2AVN', 'agentAliasName': 'Dummy', 'agentAliasArn': 'arn:aws:bedrock:us-east-1:674564987265:agent-alias/PHEBBOQFZA/UICETT2AVN', 'routingConfiguration': [], 'createdAt': datetime.datetime(2024, 1, 24, 10, 22, 12, 459839, tzinfo=tzutc()), 'updatedAt': datetime.datetime(2024, 1, 24, 10, 22, 12, 459839, tzinfo=tzutc()), 'agentAliasStatus': 'CREATING'}}


In [126]:
time.sleep(4)
#Preparing the agent

response = bedrock_agent.prepare_agent(
    agentId=aws_agent['agent'].get('agentId')
)

In [127]:
input_text:str = "Mahesh Yadav, California, 339111222333" # replace this with a prompt relevant to your agent

input_text2:str = "Calculate solar potential for 9813 Sherborne Ave, Bakersfield, CA 93311, USA with monthly bill as 300 USD"


agent_id:str = aws_agent['agent'].get('agentId') # note this from the agent console on Bedrock
agent_alias_id:str = agent_alias['agentAlias'].get('agentAliasId') # fixed for draft version of the agent
session_id:str =  str(session) # session ID

In [129]:
time.sleep(4)
generate_response_query(input_text2,agent_id,agent_alias_id,session_id)

Final answer ->
Here is the solar potential analysis for the address 9813 Sherborne Ave, Bakersfield, CA 93311, USA with a monthly electricity bill of 300 USD:

Panel Configuration: Index 30
Initial Annual Energy Production: 13199.678 kWh/year  

Remaining Lifetime Utility Bill Without Solar: 7857 USD
Federal Incentive: 6837 USD
Solar Percentage: 99.15% 
Percentage Exported to Grid: 66.36%

Leasing Option:
Leases Allowed: True
Leases Supported: True  
Annual Leasing Cost: 1544.953491211 USD
Lifetime Savings with Lease: 51227 USD 
Present Value of Lifetime Savings with Lease: 33944.83203125 USD

Cash Purchase Option: 
Out of Pocket Cost: 26299 USD
Upfront Cost: 19462 USD 
Rebate Value: 6837.740234375 USD
Payback Years: 5.25 years
Lifetime Savings with Cash Purchase: 82126 USD
Present Value of Lifetime Savings with Cash Purchase: 36319.8671875 USD

Financed Purchase Option:
Annual Loan Payment: 1544.953491211 USD  
Loan Interest Rate: 0.05
Lifetime Savings with Financed Purchase: 51227 U

In [130]:
time.sleep(4)
generate_response_query(input_text,agent_id,agent_alias_id,session_id)

Final answer ->
Thank you Mahesh Yadav for providing your contact details. I have generated a lead with your information for our solar advisors to contact you.
